# tBLASTn in arthropods

In [1]:
import time
import os

In [2]:
clustered_seq = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-CD-HIT-updated.clstr"
fasta_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-HHMER-updated.fasta"

In [3]:
with open(clustered_seq, "r") as clusters:
    c = clusters.readlines()
rep_ID_lines = [e for e in c if e.endswith("*\n")]
rep_IDs = [e.split(", >")[1].split("...")[0] for e in rep_ID_lines]
rep_IDs

['QPN36968.1',
 'UQT02520.1',
 'QMP82367.1',
 'UVF62194.1',
 'QMP82195.1',
 'QMP82285.1',
 'QMP82388.1',
 'QMP82402.1',
 'QMP82403.1',
 'QMP82112.1',
 'QMP82168.1',
 'QMP82344.1',
 'QMP82362.1',
 'QMP82338.1',
 'QPZ88437.1',
 'UMO75733.1',
 'QED21504.1',
 'UPT53723.1',
 'AVR52572.1',
 'QMP82355.1',
 'AVR52567.1',
 'UUG74181.1',
 'UMO75734.1',
 'AJG39094.1',
 'QMP82147.1',
 'QPL15381.1',
 'QMP82248.1',
 'UMO75726.1',
 'QMP82204.1',
 'QPL15288.1',
 'QPL15340.1',
 'UMO75713.1',
 'UUG74233.1',
 'QPL15368.1',
 'UMO75705.1',
 'UMO75722.1',
 'QMP82272.1',
 'URQ09139.1',
 'QIJ70032.1',
 'QMP82229.1',
 'QWC36487.1',
 'UUG74184.1',
 'AJG39084.1',
 'UMO75717.1',
 'DAZ85673.1',
 'AJG39092.1',
 'AJG39088.1',
 'QDK54862.1',
 'QEM39320.1',
 'AJG39089.1',
 'QPZ88434.1',
 'AJG39086.1',
 'QKK82920.1',
 'QPL15369.1',
 'QPZ88431.1',
 'UUG74234.1',
 'ACY56282.1',
 'UMO75729.1',
 'UYL95486.1',
 'ASR92124.1',
 'UUT42596.1',
 'QPN36942.1',
 'URY50684.1',
 'UUT42625.1',
 'AIY25030.1',
 'QOE76794.1',
 'DAZ91106

In [5]:
len(rep_IDs)

109

Search in arthropods (txid6656)

In [10]:
from Bio.Blast import NCBIWWW
for i in rep_IDs:
    #try:
    result_handle = NCBIWWW.qblast("tblastn", "nr", sequence = i, entrez_query = "txid6656[ORGN]")
    with open(f"/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/tBLASTn-PB1/{i}.xml", "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()
    #except:
        #time.sleep(5)


KeyboardInterrupt



Search in hexapods (Hexapoda, NCBI:txid6960)

In [5]:
from Bio.Blast import NCBIWWW
for i in rep_IDs:
    try:
        result_handle = NCBIWWW.qblast("tblastn", "nr", sequence = i, entrez_query = "txid6960[ORGN]")
        with open(f"/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Hexapoda-tBLASTn-PB1/{i}.xml", "w") as out_handle:
            out_handle.write(result_handle.read())
        result_handle.close()
    except:
        time.sleep(30)


KeyboardInterrupt



In [6]:
from Bio.Blast import NCBIWWW
for i in rep_IDs:
    try:
        result_handle = NCBIWWW.qblast("tblastn", "nr", sequence = i, entrez_query = "txid50557[ORGN]")
        with open(f"/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Insecta-tBLASTn-PB1/{i}.xml", "w") as out_handle:
            out_handle.write(result_handle.read())
        result_handle.close()
    except:
        time.sleep(30)


KeyboardInterrupt



Search in dicondylia (txid85512)

In [7]:
from Bio.Blast import NCBIWWW
for i in rep_IDs:
    try:
        result_handle = NCBIWWW.qblast("tblastn", "nr", sequence = i, entrez_query = "txid85512[ORGN]")
        with open(f"/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Dicondylia-tBLASTn-PB1/{i}.xml", "w") as out_handle:
            out_handle.write(result_handle.read())
        result_handle.close()
    except:
        time.sleep(30)

KeyboardInterrupt: 

## Draft

Following https://bioinf.shenwei.me/taxonkit/tutorial/#extract-all-protein-sequences-of-specific-taxons-from-the-ncbi-nr-database tutorial

Steps:

### 1. Setting up taxonkit:
1.1 Downloаd taxonkit

1.2 Downloading and decompressing taxdump.tar.gz: ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz (as recommended https://bioinf.shenwei.me/taxonkit/download/)


In [ ]:
%%bash
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz' --output ./taxdump.tar.gz

In [ ]:
%%bash
tar -zxvf taxdump.tar.gz

1.3 Copying some of the taxdump files to $HOME/.taxonkit directory

In [ ]:
%%bash
cp names.dmp nodes.dmp delnodes.dmp merged.dmp $HOME/.taxonkit

### 2. Writing Arthropoda IDs to text file.

In [11]:
%%bash
id=6656
taxonkit list --ids $id --indent "" > $id.taxid.txt
wc -l $id.taxid.txt

  925155 6656.taxid.txt


### 3. Retrieving target accessions:

3.1 Downloading prot.accession2taxid.gz file from NCBI server: /ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz.

Note: specify the name of the output file in curl command (the name of directory isn't enough).

In [ ]:
%%bash
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz' --output /Volumes/EXTERNAL/Orthomyxoviridae/prot.accession2taxid.gz

In [15]:
%%bash
cd /Volumes/EXTERNAL/Orthomyxoviridae

3.2 Retrieving target accessions (taxid -> accession numbers).

In [22]:
%%bash
pigz -dc /Volumes/EXTERNAL/Orthomyxoviridae/prot.accession2taxid.gz \
    | csvtk grep -t -f taxid -P /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.taxid.txt \
    | csvtk cut -t -f accession.version,taxid \
    | sed 1d \
    > /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc2taxid.txt

cut -f 1 /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc2taxid.txt > /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt

wc -l /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt

 19035895 /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt


### 4. Retrieving FASTA sequences from pre-formated blastdb.

4.1 Downloading nr database:

In [ ]:
%%bash
curl -O "ftp://ftp.ncbi.nih.gov/blast/db/nr.[00-100].tar.gz" --output-dir /Volumes/EXTERNAL/Orthomyxoviridae/blast

In [ ]:
%%bash
ls nr.*.tar.gz | xargs -I % tar -zxv -f % -C /Volumes/EXTERNAL/Orthomyxoviridae/blast/extracted/

Exporting nr.fa from pre-formated blastdb. Code was run in command line (couldn't activate different environment in jupyter notebook).

In [1]:
%%bash
conda activate blast
cd /Volumes/EXTERNAL/Orthomyxoviridae/blast/nr
blastdbcmd -db nr -dbtype prot -entry all -outfmt "%f" -out - | pigz -c > /Volumes/EXTERNAL/Orthomyxoviridae/nr.fa.gz


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


bash: line 3: blastdbcmd: command not found


Moved database to another portable drive. From now on, another database directory will be used.

Installing seqkit (conda install -c bioconda seqkit).

In [1]:
%%bash

time cat <(echo) <(pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.fa.gz) \
    | perl -e 'BEGIN{ $/ = "\n>"; <>; } while(<>){s/>$//;  $i = index $_, "\n"; $h = substr $_, 0, $i; $s = substr $_, $i+1; if ($h !~ />/) { print ">$_"; next; }; $h = ">$h"; while($h =~ />([^ ]+ .+?) ?(?=>|$)/g){ $h1 = $1; $h1 =~ s/^\W+//; print ">$h1\n$s";} } ' \
    | seqkit grep -f /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/6656.acc.txt -o /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz

bash: line 4: seqkit: command not found

real	0m0.107s
user	0m0.008s
sys	0m0.066s


CalledProcessError: Command 'b'\ntime cat <(echo) <(pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.fa.gz) \\\n    | perl -e \'BEGIN{ $/ = "\\n>"; <>; } while(<>){s/>$//;  $i = index $_, "\\n"; $h = substr $_, 0, $i; $s = substr $_, $i+1; if ($h !~ />/) { print ">$_"; next; }; $h = ">$h"; while($h =~ />([^ ]+ .+?) ?(?=>|$)/g){ $h1 = $1; $h1 =~ s/^\\W+//; print ">$h1\\n$s";} } \' \\\n    | seqkit grep -f /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/6656.acc.txt -o /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz\n'' returned non-zero exit status 127.

In [ ]:
ftp://ftp.ncbi.nlm.nih.gov/blast/db